In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [3]:
f = open("test_html.html", "r")
full_html = f.read()
f.close
full_html[:10]
soup = BeautifulSoup(full_html)

'<!DOCTYPE '

In [ ]:
## SCRAPING SECTION WHEN I GET THERE - CAN'T BE IN JUPYTER

#with requests_html.HTMLSession() as session:
#    resp = session.get('https://www.ncaa.com/game/3518260/play-by-play')

#resp.html.render()

#full_html = resp.html.html

In [81]:
# SET UP
columns = ['team','inning', 'half', 'outs', 'batter', 'action', 'first', 'second', 'third', 'home_score', 'visitor_score']
plays = pd.DataFrame(columns=columns)

stopwordsToCheck = ["batting starts", "failed pickoff attempt", "no play"]

home_string = "Vanderbilt"
visitor_string = "Michigan"

home = False

home_score = 0
visitor_score = 0

inning_number = 1
half = "top"

outs = 0

first = 0
second = 0
third = 0

In [82]:
# Building DataFrame
table = soup.find_all("tbody")

for inning in table[1:]:
    for row in inning.find_all("tr"):
        team = home_string if home else visitor_string
        half = "bottom" if home else "top"
        td_raw = row.find_all("td")
        play = td_raw[1].text.strip()
        if "Batting ends" in play:
            if home:
                home = False
                inning_number += 1
            else:
                home= True
            first = 0
            second = 0
            third = 0
            outs = 0
            continue
        if any(stop in play.lower() for stop in stopwordsToCheck):
            continue
        batter = play.split()[0]
        action = play.split()[1:]
        action = " ".join(action)
        runs = play.lower().count("scored") + play.lower().count("homered")
        outs += play.lower().count("out")
        first, second, third = find_basestates(play.lower(), first, second, third)
        if home:
            home_score += runs
        else:
            visitor_score += runs
        result = [team, inning_number, half, outs, batter, action, first, second, third, home_score, visitor_score]
        line_dict = dict(zip(columns, result))
        plays = plays.append(line_dict, ignore_index=True)
plays.head(10)

,team,inning,half,outs,batter,action,first,second,third,home_score,visitor_score
0,Michigan,1,top,0,Thomas,singled to left field (1-2 KFB).,1,0,0,0,0
1,Michigan,1,top,0,Franklin,singled to right field (1-0 B); Thomas advance...,1,0,0,0,0
2,Michigan,1,top,0,Brewer,"singled to left field, RBI (0-2 KS); Franklin ...",1,1,0,0,1
3,Michigan,1,top,1,Kerr,struck out swinging (1-2 BFFFS).,1,1,0,0,1
4,Michigan,1,top,2,Nelson,struck out swinging (0-2 SKS).,1,1,0,0,1
5,Michigan,1,top,3,Bullock,struck out swinging (0-2 KSS).,1,1,0,0,1
6,Vanderbilt,1,bottom,1,Martin,struck out looking (2-2 BSBFK).,0,0,0,0,1
7,Vanderbilt,1,bottom,2,Bleday,flied out to cf (0-0).,0,0,0,0,1
8,Vanderbilt,1,bottom,2,Paul,walked (3-1 BBBKB).,0,0,0,0,1
9,Vanderbilt,1,bottom,3,Clarke,flied out to lf (0-0).,0,0,0,0,1


In [80]:
# Utility Funcitions
def find_basestates(play, first, second, third):
    actions = play.split(";")
    actions = [a.strip() for a in actions]
    first, second, third = first, second, third
    for action in actions:
        if "singled" in action:
            first = 1
        if "doubled" in action:
            second = 1
        if "tripled" in action:
            third = 1
        if "advanced" in action or "stole" in action:
            base = action.split()[-1]
            if base == "first":
                first = 1
            if base == "second":
                second = 1
            if base == "third":
                third = 1
    return first, second, third

In [61]:
plays["action"].iloc[2]

'singled to left field, RBI (0-2 KS); Franklin advanced to second; Thomas scored.'